# GEOG696C Spatiotemporal Data Analytics: Term Project
## extractPrecipTimeSeries.ipynb
This script extracts time series of precipitation per pixel from the CHIRPS dataset, via Google Earth Engine.

In [125]:
from pathlib import Path
import os
import sys
import pandas as pd
# import geopandas as gpd
import numpy as np
import rasterio as rio
from rasterio.plot import show
from rasterio.mask import mask
# import rioxarray
# import xarray
# from shapely.geometry import Polygon
# from shapely.geometry import Point
# import matplotlib.pyplot as plt
# from itertools import chain
from datetime import datetime, timezone, timedelta

In [22]:
# Set the root path
# rootPath = Path('/media/mule/Projects/NASA/NIP/Data')
rootPath = Path('C:/Users/alexsaunders/Documents/01_uoa/01_study/2023/geog696c/project/data')

In [4]:
import ee

In [5]:
ee.Authenticate()

Enter verification code:  4/1AfJohXmhves3bz_I47CM0Xf5ASXnsRPL5gaP3-xAAk7ArIgD2rlqovqDrGI



Successfully saved authorization token.


In [6]:
ee.Initialize()

## 1) Get the CHIRPS Pentad (i.e. 5-day) data

In [186]:
# Define region of interest (ROI)
roi = ee.FeatureCollection('projects/ee-nip/assets/BangladeshBB')

In [187]:

# dates = pd.date_range(startDate, endDate, freq='5D')

In [188]:
dates

DatetimeIndex(['2001-01-01', '2001-01-06', '2001-01-11', '2001-01-16',
               '2001-01-21', '2001-01-26', '2001-01-31', '2001-02-05',
               '2001-02-10', '2001-02-15',
               ...
               '2022-11-17', '2022-11-22', '2022-11-27', '2022-12-02',
               '2022-12-07', '2022-12-12', '2022-12-17', '2022-12-22',
               '2022-12-27', '2023-01-01'],
              dtype='datetime64[ns]', length=1608, freq='5D')

In [189]:
# Calculate time series on pentad data

# Get pentad data for dates and roi
chirps = ee.ImageCollection('UCSB-CHG/CHIRPS/PENTAD')
scale = 5566 # 0.5 degs

# Get filtered image collection
startDate = '2001-01-01'
endDate = '2023-01-01'
filtered = chirps.filter(ee.Filter.date(startDate, endDate)).filterBounds(roi)

# Image properties - dates
dateTime = np.array(filtered.aggregate_array('system:time_start').getInfo())
month = np.array(filtered.aggregate_array('month').getInfo())
pentad = np.array(filtered.aggregate_array('pentad').getInfo())

# Get the geometry (same for all images)
img = filtered.first()
fc = img.sample(region=roi, scale=scale, geometries=True)
vals = fc.toList(fc.size()).getInfo() # list of dictionaries
lons=[d['geometry']['coordinates'][0] for d in vals if 'geometry' in d]
lats=[d['geometry']['coordinates'][1] for d in vals if 'geometry' in d]
coords = pd.DataFrame(data=[lons, lats], index=['lat','lon']).T

In [190]:
coords

,lat,lon
0,88.026226,26.624893
1,88.076226,26.624893
2,88.126226,26.624893
3,88.176227,26.624893
4,88.226227,26.624893
...,...,...
9312,92.426246,20.674866
9313,92.476246,20.674866
9314,92.526247,20.674866
9315,92.576247,20.674866


In [192]:
dates = pd.to_datetime(dateTime, unit='ms')

In [193]:
dates

DatetimeIndex(['2001-01-01', '2001-01-06', '2001-01-11', '2001-01-16',
               '2001-01-21', '2001-01-26', '2001-02-01', '2001-02-06',
               '2001-02-11', '2001-02-16',
               ...
               '2022-11-11', '2022-11-16', '2022-11-21', '2022-11-26',
               '2022-12-01', '2022-12-06', '2022-12-11', '2022-12-16',
               '2022-12-21', '2022-12-26'],
              dtype='datetime64[ns]', length=1584, freq=None)

In [ ]:
# Loop through in small time chunks and get and record precip value
ntimes=len(dateTime)
nlocs=len(coords)
precipValsAllDates=np.zeros([ntimes, nlocs])
for i, date in enumerate(dates): 
    start = ee.Date(date)
    end = ee.Date(pd.to_datetime(date)+timedelta(days=2))
    filtered = chirps.filter(ee.Filter.date(start, end)).filterBounds(roi)
    
    # Take the first image and process it
    img = filtered.first()
    fc = img.sample(region=roi, scale=scale, geometries=True)
    vals = fc.toList(fc.size()).getInfo() # list of dictionaries

    # Get the precip vals
    precipVals = [d['properties']['precipitation'] for d in vals if 'properties' in d]
    
    # Save image values to array
    precipValsAllDates[i,:] = precipVals

In [ ]:
precipValsAllDates